In [1]:
import pandas as pd
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import requests
import json

from robot_20220306_cpu_backup import Robot

In [2]:
robot = Robot()

start to load data
finish loading data
start to load models
finish loading translation model
finish loading decisionMakingNet
finish loading suggestionNet
finish loading diagnoseNet


D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
quit = False
#robot initialize
robot.reset()
action = input('欢迎使用robot doctor, 请问您觉得哪里不舒服?')
symptom = action.strip()

# input the dirst symptom
robot.input_sid((robot.cname2id[symptom])) # there is no relations
if robot.is_enough():
    #inference
    print(robot.inference_disease_with_probs())
else:
    while not quit:
        #10 symptoms loop
        suggestions, probs = robot.suggestion(10)
        
        for suggestion in suggestions:
            #suggestion_id = robot.index2suggestion_id[suggestion]
            suggestion_name = robot.id2cname[suggestion]
            question = suggestion_name + '?'
            node_id = suggestion

            #ask question
            #print(question)
            if node_id in robot.wrong_set:
                action = 'n'
                print('has answered before, answer is:{}'.format(action))
            elif node_id in robot.selected_set:
                action = 'y'
                print('has answered before, answer is:{}'.format(action))
            else:
                print('asking this node:{}'.format(node_id))
                action = input(question)
            if action.strip() == 'y':
                robot.input_sid((node_id))
                quit = robot.is_enough()
#                 print(robot.inference_disease_with_probs())
                break
            else:
                robot.update_wrong_sid([node_id])
                quit = robot.is_enough()
#                 print(robot.inference_disease_with_probs())
                if quit:
                    break
            continue
                
    #infer disease
    print('经过思考作出以下诊断:')
    print(robot.inference_disease_with_probs())
    print('推荐科室:')
    print(robot.recommend_departments())

欢迎使用robot doctor, 请问您觉得哪里不舒服? 腹部不适


D:\workfiles\codes\rasa\task-oriented-dialogue-system-for-smart-follow-up\nlu\actions\robot_20220322\robot_20220306_cpu_backup.py:576: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(x, dtype=torch.float)


asking this node:129


消化不良? y


asking this node:9


...

asking this node:362


嗳气? n


asking this node:14


恶心? y


经过思考作出以下诊断:
(['劳力性心绞痛', '慢性胃炎', '慢性扁桃体炎', '慢性胆囊炎', '慢性鼻咽炎', '疱疹样湿疹', '肺炎克雷白杆菌肺炎', '胃溃疡', '小儿丘疹性肢皮炎综合征', '慢性阿米巴性痢疾'], (0.49757360591173055, 0.21811318055294696, 0.13260086196361753, 0.0217745483963391, 0.020033828311428636, 0.015332332641826934, 0.014257727710909133, 0.011509495157993169, 0.009878671943578863, 0.007486562152634201))
推荐科室:
('皮肤科', '消化科', '感染科', '呼吸内科', '耳鼻喉科', '耳鼻喉-头颈外科', '泌尿外科', '心内科')


In [4]:
symptom_entity_list = [robot.id2entity[i] for i in robot.selected_index]
[robot.diagnose_net.eentity2ename[_item] for _item in symptom_entity_list]

['头痛', '乏力', '畏寒', '抽搐', '浑身酸痛', '高热', '咽喉痛']